In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import keras
import tensorflow as tf
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras import backend as K
from keras.layers.normalization import BatchNormalization

from tensorflow.keras.optimizers import Adam, RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from keras.wrappers.scikit_learn import KerasClassifier
from keras.applications.resnet50 import ResNet50
from keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

In [ ]:
train_data=tf.keras.preprocessing.image_dataset_from_directory('/content/drive/MyDrive/chest',image_size=(224,224),color_mode="grayscale",
    batch_size=24,
    shuffle=True,
    seed=123)


Found 324 files belonging to 2 classes.


In [ ]:
def conv2d_block(input_tensor,n_filters,kernel_size=3):
  x=input_tensor
  for i in range(2):
    x=tf.keras.layers.Conv2D(filters=n_filters,kernel_size=(kernel_size,kernel_size),padding='same')(x)
    x=tf.keras.layers.Activation('relu')(x)
  return x
def encoder_block(inputs,n_filter,pool_size,dropout):
  f=conv2d_block(input_tensor=inputs,n_filters=n_filter)
  p=tf.keras.layers.MaxPooling2D(pool_size)(f)
  p=tf.keras.layers.Dropout(dropout)(p)
  return f,p
def encoder(inputs):
  f1,p1=encoder_block(inputs,n_filter=64,pool_size=(2,2),dropout=0.3)
  f2,p2=encoder_block(p1,n_filter=128,pool_size=(2,2),dropout=0.3)
  f3,p3=encoder_block(p2,n_filter=256,pool_size=(2,2),dropout=0.3)
  f4,p4=encoder_block(p3,n_filter=512,pool_size=(2,2),dropout=0.3)
  return p4,(f1,f2,f3,f4)
def bottleneck(inputs):
  bottle_neck=conv2d_block(inputs,n_filters=1024)
  return bottle_neck
def decoder_block(inputs,conv_output,n_filter,kernel_size,strides,dropout):
  u=tf.keras.layers.Conv2DTranspose(n_filter,kernel_size,strides=strides,padding='same')(inputs)
  c=tf.keras.layers.concatenate([u,conv_output])
  c = tf.keras.layers.Dropout(dropout)(c)
  c=conv2d_block(c,n_filter,kernel_size=3)
  return c
def decoder(inputs,convs):
  f1,f2,f3,f4=convs
  c6=decoder_block(inputs,f4,n_filter=512,kernel_size=(3,3),strides=(2,2),dropout=0.3)
  c7=decoder_block(c6,f3,n_filter=256,kernel_size=(3,3),strides=(2,2),dropout=0.3)
  c8=decoder_block(c7,f2,n_filter=128,kernel_size=(3,3),strides=(2,2),dropout=0.3)
  c9=decoder_block(c8,f1,n_filter=64,kernel_size=(3,3),strides=(2,2),dropout=0.3)
  outputs=tf.keras.layers.Conv2D(32,kernel_size=(1,1),activation='relu')(c9)
  outputs=tf.keras.layers.Flatten()(outputs)
  #outputs=tf.keras.layers.Dense(16, activation='relu', bias_regularizer=tf.keras.regularizers.L2(1e-4))(outputs)
  outputs=tf.keras.layers.Dense(1,activation='sigmoid',bias_regularizer=tf.keras.regularizers.L2(1e-4))(outputs)
  return outputs
def unet():
  inputs=tf.keras.layers.Input(shape=(224,224,1,))
  encoder_output,convs=encoder(inputs)
  bottle_neck=bottleneck(encoder_output)
  outputs=decoder(bottle_neck,convs)
  model=tf.keras.Model(inputs=inputs,outputs=outputs)
  return model
def main():
  model = unet()
  opt = Adam(lr=0.001, decay=1e-6)
  model.compile(optimizer=opt, loss='binary_crossentropy',metrics=['accuracy'])
  batch_size=24
  
  model.fit(x=train_data,epochs=20)
  model.save('chest.h5')
  model.summary()

In [ ]:
main()

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/20
14/14 [==============================] - 117s 3s/step - loss: 301.9738 - accuracy: 0.4599
Epoch 2/20
14/14 [==============================] - 14s 960ms/step - loss: 9.0184 - accuracy: 0.6944
Epoch 3/20
14/14 [==============================] - 14s 965ms/step - loss: 2.8378 - accuracy: 0.8210
Epoch 4/20
14/14 [==============================] - 14s 973ms/step - loss: 0.6199 - accuracy: 0.8796
Epoch 5/20
14/14 [==============================] - 14s 985ms/step - loss: 0.3628 - accuracy: 0.8827
Epoch 6/20
14/14 [==============================] - 14s 994ms/step - loss: 0.2991 - accuracy: 0.8858
Epoch 7/20
14/14 [==============================] - 14s 1s/step - loss: 0.2291 - accuracy: 0.9012
Epoch 8/20
14/14 [==============================] - 14s 1s/step - loss: 0.2016 - accuracy: 0.9198
Epoch 9/20
14/14 [==============================] - 14s 1s/step - loss: 0.1590 - accuracy: 0.9414
Epoch 10/20
14/14 [==============================] - 15s 1s/step - loss: 0.1615 - accuracy: 0.9383
E

In [ ]:
from keras.models import load_model
model = load_model('/content/chest.h5')
import cv2
import numpy as np
from tensorflow.keras.preprocessing import image

img = image.load_img('/content/drive/MyDrive/download (1).jpg', target_size=(224,224),color_mode='grayscale')
#img = PIL.Image.open("/content/drive/MyDrive/download (1).jpg")
#newsize = (224, 224)
#image = cv2.resize(img, (224,224))
#image = np.asarray(image).reshape(-1,224,224,1)
#image = image[None,:,:,1]
#img = np.asarray(img).reshape(-1,224,224,1)
img = image.img_to_array(img)
prediction = model.predict(img)

In [ ]:
prediction

array([[2.7531520e-08],
       [1.4734665e-03],
       [1.3023447e-03]], dtype=float32)